**DEFINING INPUT FILE**

In [ ]:
model_dir = 'Full_Area_S2_2016_v2/FULLAREA_S2_2016_v2/trained_model_0,1LR_200epochs_0,95DR_WCE'
train_tfrecord = 'Full_Area_S2_2016_v2/FULLAREA_S2_2016_v2/dataset2/def_dataset_train.tfrecord'
test_tfrecord = 'Full_Area_S2_2016_v2/FULLAREA_S2_2016_v2/dataset2/def_dataset_test.tfrecord'

**IMPORT LIBRARIES**

In [2]:
import numpy as np
import gdal
import os
import deepgeo
import deepgeo.networks.model_builder as mb
import deepgeo.networks.loss_functions as lossf


**SETTING UP MODEL PARAMETERS**

In [3]:
weights_train = lossf.compute_weights_mean_proportion(train_tfrecord, ['no_data', 'cloud', 'bare land', 'natural forest', 'other land cover','oil palm', 'water'] , classes_zero =['no_data']) #consider 0 for no data in the network
weights_eval = lossf.compute_weights_mean_proportion(test_tfrecord, ['no_data', "cloud", 'bare land' ,'natural forest', 'other land cover','oil palm', 'water'] , classes_zero=['no_data'])

In [ ]:
params = {
    'network': 'unet',
    'epochs':200 ,
    'batch_size': 5,
    'chip_size': 300,
    'bands': 9,
    'learning_rate': 0.1,
    'learning_rate_decay': True,
    'decay_rate': 0.95,
    'l2_reg_rate': 0.0005,
    'chips_tensorboard': 2,
    'loss_func': 'weighted_cross_entropy',#avg_soft_dice
    'data_aug_ops': ['rot90', 'rot180', 'rot270', 'flip_left_right',
                     'flip_up_down', 'flip_transpose'],
    'data_aug_per_chip': 6,
    'class_weights' :{'train':weights_train, 'eval':weights_eval},
    'num_classes': 7,
    'class_names': ['no_data', 'cloud' ,'bare land', 'natural forest', 'other land cover','oil palm', 'water'],
    'num_compositions': 1,
    'bands_plot': [1, 5, 2],
    'Notes': 'Full_Sentinel2_2016.'
}

**RUN THE U-NET MODEL**

In [ ]:
model = mb.ModelBuilder(params)
model.train(train_tfrecord, test_tfrecord, model_dir)